In [45]:
# 参照するディレクトリのパス追加プロセス
import sys
import pprint
import os

pprint.pprint(sys.path)

['/Users/takahashishun/mlfin/labeling',
 '/Users/takahashishun/opt/anaconda3/lib/python39.zip',
 '/Users/takahashishun/opt/anaconda3/lib/python3.9',
 '/Users/takahashishun/opt/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/Users/takahashishun/opt/anaconda3/lib/python3.9/site-packages',
 '/Users/takahashishun/opt/anaconda3/lib/python3.9/site-packages/aeosa',
 '/Users/takahashishun/opt/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg',
 '/Users/takahashishun/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions',
 '/Users/takahashishun/.ipython',
 '/Users/takahashishun/mlfin',
 '/Users/takahashishun',
 '/Users/takahashishun']


In [46]:
# mlfinがパスディレクトリに追加されているかどうかは重要
import numpy as np
import pandas as pd

# Snippet 3.1, page 44, Daily Volatility Estimates
from mlfin.util.multiprocess import *
from mlfin.util.volatility import get_daily_vol
from mlfin.labeling.labeling import *
from mlfin.filters.filters import *


In [47]:
import math

import ccxt
from crypto_data_fetcher.gmo import GmoFetcher
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
import seaborn as sns
import datetime
import time

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score, KFold, TimeSeriesSplit

In [48]:
# 同階層に保存されているエンコードテキスト

df = joblib.load("temp.txt")
df

,op,hi,lo,cl,volume
timestamp,,,,,
2018-09-05 08:05:00+00:00,818724.0,818724.0,818724.0,818724.0,0.01
2018-09-05 08:20:00+00:00,821250.0,821250.0,821250.0,821250.0,0.01
2018-09-05 08:45:00+00:00,819260.0,819282.0,819260.0,819282.0,0.02
2018-09-05 08:50:00+00:00,819765.0,819765.0,819765.0,819765.0,0.01
2018-09-05 09:00:00+00:00,820000.0,820000.0,819500.0,819500.0,0.02
...,...,...,...,...,...
2021-03-31 23:35:00+00:00,6520010.0,6522172.0,6514990.0,6518843.0,7.31
2021-03-31 23:40:00+00:00,6519427.0,6520989.0,6513263.0,6518492.0,7.35
2021-03-31 23:45:00+00:00,6519089.0,6528367.0,6517671.0,6524820.0,6.05


In [49]:
close = df['cl'].copy()
vol = get_daily_vol(close)
vol

timestamp
2018-09-06 08:50:00+00:00         NaN
2018-09-06 09:00:00+00:00    0.003038
2018-09-06 09:15:00+00:00    0.003205
2018-09-06 09:20:00+00:00    0.002758
2018-09-06 09:30:00+00:00    0.003737
                               ...   
2021-03-31 23:35:00+00:00    0.007515
2021-03-31 23:40:00+00:00    0.007447
2021-03-31 23:45:00+00:00    0.007385
2021-03-31 23:50:00+00:00    0.007312
2021-03-31 23:55:00+00:00    0.007239
Name: cl, Length: 256140, dtype: float64

In [50]:
# なぜかfiltersに組み込めないが、競合とか？
def getTEvents(gRaw:pd.Series, h):
  

    tEvents, sPos, sNeg = [], 0, 0
    diff = gRaw.diff()
    for i in diff.index[1:]:
        sPos, sNeg = max(0, sPos + diff.loc[i]), min(0, sNeg + diff.loc[i])
        if sNeg < -h:
            sNeg = 0
            tEvents.append(i)
        elif sPos > h:
            sPos = 0
            tEvents.append(i)
    
    return pd.DatetimeIndex(tEvents)

In [51]:
tEvents = getTEvents(vol, 0.05)
tEvents

DatetimeIndex(['2018-11-15 20:30:00+00:00', '2018-11-18 16:05:00+00:00',
               '2018-11-21 08:50:00+00:00', '2018-11-22 19:30:00+00:00',
               '2018-11-25 01:20:00+00:00', '2018-11-25 22:20:00+00:00',
               '2018-11-26 19:25:00+00:00', '2019-01-09 11:25:00+00:00',
               '2019-03-14 08:30:00+00:00', '2019-04-02 05:20:00+00:00',
               '2019-04-02 20:40:00+00:00', '2019-04-03 05:45:00+00:00',
               '2019-04-03 22:15:00+00:00', '2019-05-13 15:40:00+00:00',
               '2019-05-14 06:45:00+00:00', '2019-05-26 07:35:00+00:00',
               '2019-06-26 21:30:00+00:00', '2019-06-29 14:15:00+00:00',
               '2019-07-16 19:05:00+00:00', '2019-07-21 12:00:00+00:00',
               '2019-09-24 20:20:00+00:00', '2019-09-27 02:55:00+00:00',
               '2019-10-25 17:15:00+00:00', '2019-10-26 14:30:00+00:00',
               '2019-10-27 21:00:00+00:00', '2020-03-12 11:05:00+00:00',
               '2020-03-13 11:50:00+00:00', '2020-0

In [ ]:
apply_pt_sl_on_t1